# Real-world network

In [1]:
import networkx as nx
from networkx.algorithms.centrality import *
import numpy as np
import operator
from analysis import *

In [2]:
g = nx.read_gml("./RealNetworks/football.gml")

In [3]:
print(nx.info(g))

Graph with 115 nodes and 613 edges


<ipython-input-3-99dac2172e06>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))


# Check connectedness

In [4]:
nx.is_connected(g)

True

In [5]:
nx.number_connected_components(g)

1

# Get LCC if graph is disconnected

# Relabel it all if nodes are strings
- To be able to run all types of correlation together
- But put the graph relabled in `g_relabled`

In [6]:
from myTruss import mappingAndRelabeling

In [7]:
g_relabled = mappingAndRelabeling(g)

In [8]:
print(nx.info(g))
print(nx.info(g_relabled))

Graph with 115 nodes and 613 edges
Graph with 115 nodes and 613 edges


<ipython-input-8-632c4a2d19f6>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))
<ipython-input-8-632c4a2d19f6>:2: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g_relabled))


In [9]:
g = g_relabled

# Infer community structure with Louvain

In [10]:
nx.__version__

'2.8.4'

In [11]:
import pandas as pd
from os.path  import exists
import infomap as im

In [12]:
im.__version__

'2.3.0'

In [13]:
G = g

In [14]:
partition   = nx.algorithms.community.louvain_communities(G, seed = 0)
partition_Q = nx.algorithms.community.modularity(G, partition)

for seed in Bar("Progress", check_tty = False).iter(range(1,5)):
    partition_   = nx.algorithms.community.louvain_communities(G, seed = seed)
    partition_Q_ = nx.algorithms.community.modularity(G, partition_)
    
    if partition_Q_ > partition_Q:
        partition   = partition_
        partition_Q = partition_Q_

Progress |################################| 4/4


In [15]:
communities = { community_ID : set(community) for (community_ID, community) in enumerate(partition, start = 1)}

In [16]:
community_assignments = dict()
for (community_ID, community) in communities.items():
    for node in community:
        community_assignments[node] = community_ID

In [17]:
results_file = "./results/football-modularity.csv"

if not exists(results_file):
    # Map Equation Centrality
    tree_file = "./results/football-temp.tree"

    # use Infomap to write a tree file that we change to reflect the Louvain partition
    infomap = im.Infomap(silent = True, num_trials = 1, directed = False, two_level = True, seed = 42)
    infomap.add_networkx_graph(G)
    infomap.run()
    infomap.write_tree(tree_file)
    
    infomap_tree = dict()

    # read the tree file
    with open(tree_file, "r") as fh:
        for line in fh:
            if not line.startswith("#"):
                _, flow, name, node_id = line.strip().split(" ")
                name = name[1:-1]
                infomap_tree[int(name)] = (flow, node_id)
    
    # write the Louvain partition to an Infomap tree file
    with open("./results/football-modularity.tree", "w") as fh:
        for community_ID, community in enumerate(partition, start = 1):
            for node_pos, node in enumerate(community, start = 1):
                path = f"{community_ID}:{node_pos}"
                flow, node_id = infomap_tree[node]
                fh.write(f"{path} {flow} \"{node}\" {node_id}\n")
    
    # Load the Louvain partition with Infomap and use unrecorded link teleportation
    infomap2 = im.Infomap("--silent --no-infomap --cluster-data ./results/football-modularity.tree")
    node_mapping = infomap2.add_networkx_graph(G)
    infomap2.run()
    
    mec_ranks = [node for node, _score in sorted( [ (node_mapping[node.node_id], node.modular_centrality) for node in infomap2.iterLeafNodes() ]
                                                 , key     = second
                                                 , reverse = True
                                                 )]

    
    index = []
    for node in G.nodes:
        index.append(node)

    results = pd.DataFrame(index=index)

    results["map equation centrality rank"]       = toRanking(mec_ranks,        results.index)
    results.to_csv(results_file)

else:
    results = pd.read_csv(results_file, index_col = 0)

    mec_ranks         = mkRanks(results["map equation centrality rank"])

In [18]:
len(mec_ranks)

115

In [19]:
# node 107 has rank 0 (i.e. 1)
# node 1684 has rank 1 (i.e. 2)
# ...
# node 0 has rank 4 (i.e. 5)
mec_ranks

[0,
 104,
 3,
 5,
 2,
 6,
 15,
 7,
 67,
 53,
 88,
 1,
 65,
 70,
 76,
 17,
 20,
 87,
 27,
 62,
 34,
 38,
 43,
 18,
 61,
 31,
 4,
 69,
 9,
 16,
 23,
 98,
 40,
 72,
 74,
 81,
 84,
 32,
 64,
 100,
 39,
 106,
 13,
 47,
 60,
 68,
 8,
 77,
 78,
 111,
 51,
 21,
 22,
 35,
 79,
 80,
 82,
 19,
 30,
 55,
 29,
 73,
 110,
 46,
 49,
 114,
 83,
 37,
 109,
 45,
 89,
 25,
 66,
 91,
 44,
 48,
 86,
 92,
 96,
 113,
 56,
 58,
 95,
 99,
 71,
 12,
 14,
 54,
 26,
 41,
 11,
 24,
 93,
 102,
 10,
 107,
 52,
 108,
 101,
 94,
 33,
 103,
 105,
 75,
 112,
 57,
 63,
 85,
 50,
 90,
 28,
 97,
 59,
 36,
 42]

In [20]:
counter = len(mec_ranks)
dict_map_eq_centrality = {}

In [21]:
for node in mec_ranks:
    dict_map_eq_centrality[int(node)] = counter
    counter = counter - 1

In [22]:
dict_map_eq_centrality

{0: 115,
 104: 114,
 3: 113,
 5: 112,
 2: 111,
 6: 110,
 15: 109,
 7: 108,
 67: 107,
 53: 106,
 88: 105,
 1: 104,
 65: 103,
 70: 102,
 76: 101,
 17: 100,
 20: 99,
 87: 98,
 27: 97,
 62: 96,
 34: 95,
 38: 94,
 43: 93,
 18: 92,
 61: 91,
 31: 90,
 4: 89,
 69: 88,
 9: 87,
 16: 86,
 23: 85,
 98: 84,
 40: 83,
 72: 82,
 74: 81,
 81: 80,
 84: 79,
 32: 78,
 64: 77,
 100: 76,
 39: 75,
 106: 74,
 13: 73,
 47: 72,
 60: 71,
 68: 70,
 8: 69,
 77: 68,
 78: 67,
 111: 66,
 51: 65,
 21: 64,
 22: 63,
 35: 62,
 79: 61,
 80: 60,
 82: 59,
 19: 58,
 30: 57,
 55: 56,
 29: 55,
 73: 54,
 110: 53,
 46: 52,
 49: 51,
 114: 50,
 83: 49,
 37: 48,
 109: 47,
 45: 46,
 89: 45,
 25: 44,
 66: 43,
 91: 42,
 44: 41,
 48: 40,
 86: 39,
 92: 38,
 96: 37,
 113: 36,
 56: 35,
 58: 34,
 95: 33,
 99: 32,
 71: 31,
 12: 30,
 14: 29,
 54: 28,
 26: 27,
 41: 26,
 11: 25,
 24: 24,
 93: 23,
 102: 22,
 10: 21,
 107: 20,
 52: 19,
 108: 18,
 101: 17,
 94: 16,
 33: 15,
 103: 14,
 105: 13,
 75: 12,
 112: 11,
 57: 10,
 63: 9,
 85: 8,
 50: 7,
 

In [23]:
# Write Centrality
with open('./MapEquationCentralityResults/Misc-Football/dict_map_eq_centrality.txt', 'w') as f:
    for key, value in dict_map_eq_centrality.items():
        f.write('%s:%s\n' % (key, value))

In [24]:
# Read Centrality
dict_map_eq_centrality_read = dict()
with open('./MapEquationCentralityResults/Misc-Football/dict_map_eq_centrality.txt') as raw_data:
    for item in raw_data:
        if ':' in item:
            key,value = item.split(':', 1)
            value = value.replace('\n', '')
            dict_map_eq_centrality_read[int(key)]=float(value)
        else:
            pass # deal with bad lines of text here

In [25]:
dict_map_eq_centrality_read == dict_map_eq_centrality

True